# Install Dependences

In [ ]:
!pip install tensorflow==2.18.0
!pip install tensorflow-tpu==2.18.0 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html
!git clone https://github.com/RR-project-group/TC-AcceleratedMonteCarloSimulation.git

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/239.6 MB 7.6 MB/s eta 0:00:22^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://storage.googleapis.com/libtpu-tf-releases/index.html
ERROR: Could not find a version that satisfies the requirement tensorflow-tpu==2.18.0 (from versions: 0.0.1rc0)
ERROR: No matching distribution found for tensorflow-tpu==2.18.0


# Import library

In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
import os
os.chdir('TC-AcceleratedMonteCarloSimulation')
import lmm
import bs

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/zhangdanfeng/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/p5/7lvp73g10bv9_6jcqzvcs6km0000gn/T/ipykernel_9395/4106891004.py", line 1, in <module>
    import tensorflow as tf
ModuleNotFoundError: No module named 'tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/zhangdanfeng/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zhangdanfeng/anaconda3/lib/python3.12/site-packages/IPython/core/ultratb.py", line 1454, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zhangdanfeng/anaconda3/lib/pyt

# Detect ColabTPU


# ===== Detect TPU or fallback to CPU/GPU =====
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    tpu_strategy = tf.distribute.TPUStrategy(resolver)
    print("✅ TPU Detected.")
    tpu_available = True
except:
    tpu_strategy = tf.distribute.get_strategy()
    print("⚠️ TPU Not Found, running on CPU/GPU.")
    tpu_available = False

# BS simulation

In [ ]:

# ===== Parameter Configuration =====
n_steps_list = [25, 50, 100]

# ===== Run Experiment in Specific Strategy Context =====
def run_experiment(dtype, n_steps, label):
        # tf.keras.backend.clear_session()
        start = time.time()
        price = run_bs_simulation(n_paths, n_steps, dtype=dtype, seed=seed)
        end = time.time()
        return {
            "label": label,
            "dtype": str(dtype.name),
            "n_steps": n_steps,
            "price": float(price.numpy()),
            "time": round(end - start, 3)
        }

# ===== Run all configurations =====
results = []

# CPU/GPU: float32 only
for n_steps in n_steps_list:
    result = run_experiment(tf.float32, n_steps, "CPU-float32")
    results.append(result)

# TPU tests: float32 and bfloat16


if tpu_available:
    for n_steps in n_steps_list:
        tf.keras.backend.clear_session()
        with tpu_strategy.scope():
            result = run_experiment(tf.bfloat16, n_steps, f"TPU-{dtype.name}")
        results.append(result)
        print(n_steps)

# ===== Display Results =====
df = pd.DataFrame(results)
print("\n📊 Benchmark Results:")
print(df)

# ===== Optional: Plotting =====
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Execution Time Plot
for key, grp in df.groupby('label'):
    ax[0].plot(grp['n_steps'], grp['time'], marker='o', label=key)
ax[0].set_title("Execution Time vs. Time Steps")
ax[0].set_xlabel("Time Steps")
ax[0].set_ylabel("Time (s)")
ax[0].legend()
ax[0].grid(True)

# Option Price Plot
for key, grp in df.groupby('label'):
    ax[1].plot(grp['n_steps'], grp['price'], marker='o', label=key)
ax[1].set_title("Estimated Price vs. Time Steps")
ax[1].set_xlabel("Time Steps")
ax[1].set_ylabel("Option Price")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()